## Repository selection
We want to list and select repositories high number of CVEs references to them.   

In [1]:
from dotenv import load_dotenv
import os

os.chdir("..")
os.getcwd()
load_dotenv()

print(os.getcwd())

/Users/daniele/rise/Thesis-daniele/Vulnerability_prediction


In [2]:
import pandas as pd
from  datasets import load_dataset

ds = load_dataset('Eathus/cve-references-list', split='train')
references_df = ds.to_pandas()

ds = load_dataset('Eathus/filtered-vulnerabilities', split='train')
vulnerabilities_df = ds.to_pandas()

/home/edvinn/KTH/DA231X-ex/mthesis_pyenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ref_df = references_df[references_df['url'].str.contains('github.*issues')]
ref_df = ref_df[pd.to_datetime(ref_df.published) > pd.to_datetime('2023-05-01')]

In [4]:
import re 
from tqdm import tqdm

repos = set()

for id, url in tqdm(ref_df[['id','url']].itertuples(index=False)):
    
    pattern = r'https://github.com/([^/]+)/([^/]+)/issues/'
    match = re.match(pattern, url)
    if match:
        owner = match.group(1)
        repo = match.group(2)
        repos.add((owner, repo, id))
        
print(len(repos))

1696it [00:00, 64952.56it/s]

1601


In [5]:
df = pd.DataFrame(repos, columns=['owner', 'repo_name', 'cve'])
df_group = df.groupby(['owner', 'repo_name']).count()
df_group_sorted = df_group.sort_values(by='cve', ascending=False)
top_50_groups = df_group_sorted.head(50)
top_50_groups.head(50)
display(df_group_sorted.head(50))

,,cve
owner,repo_name,
openlink,virtuoso-opensource,33
gpac,gpac,32
ppp-src,CVE,27
CveSecLook,cve,23
Hebing123,cve,21
jerryscript-project,jerryscript,18
LvZCh,td,17
Hacker0xone,CVE,16
ppp-src,a,16


In [6]:
repos_set = set(top_50_groups.index)
display(repos_set)

{('ChurchCRM', 'CRM'),
 ('CveSecLook', 'cve'),
 ('Hacker0xone', 'CVE'),
 ('Hebing123', 'cve'),
 ('HryspaHodor', 'CVE'),
 ('ImageMagick', 'ImageMagick'),
 ('Lanxiy7th', 'lx_CVE_report-'),
 ('LibreDWG', 'libredwg'),
 ('LvZCh', 'td'),
 ('MiczFlor', 'RPi-Jukebox-RFID'),
 ('MonetDB', 'MonetDB'),
 ('Num-Nine', 'CVE'),
 ('OpenRapid', 'rapidcms'),
 ('Piwigo', 'Piwigo'),
 ('WebAssembly', 'wabt'),
 ('anhdq201', 'netbox'),
 ('axiomatic-systems', 'Bento4'),
 ('cesanta', 'mjs'),
 ('chakra-core', 'ChakraCore'),
 ('daylightstudio', 'FUEL-CMS'),
 ('dromara', 'hutool'),
 ('emlog', 'emlog'),
 ('emqx', 'nanomq'),
 ('espeak-ng', 'espeak-ng'),
 ('free5gc', 'free5gc'),
 ('funadmin', 'funadmin'),
 ('gpac', 'gpac'),
 ('greenpau', 'caddy-security'),
 ('hwchase17', 'langchain'),
 ('jeecgboot', 'jeecg-boot'),
 ('jerryscript-project', 'jerryscript'),
 ('kubernetes', 'kubernetes'),
 ('libming', 'libming'),
 ('llvm', 'llvm-project'),
 ('magic2353112890', 'cve'),
 ('matthiaskramm', 'swftools'),
 ('micropython', 'mic

## Select all the issues/pull form  CVEs references that are linked to one of those repos 

In [7]:
filtered_references_df_tmp = references_df[
    references_df['url'].str.contains('github.com/[^/]+/[^/]+/issues/[0-9]+', regex=True)].copy()


def funct(url):
    pattern = r'https://github.com/([^/]+)/([^/]+)/'
    match = re.match(pattern, url)
    if match:
        owner = match.group(1)
        repo = match.group(2)
        return (owner, repo)
    return None


filtered_references_df_tmp['owner_repo'] = filtered_references_df_tmp['url'].map(funct)
filtered_references_df = filtered_references_df_tmp[filtered_references_df_tmp.owner_repo.isin(repos_set)]
filtered_references_df.sample(5)

,id,published,url,tags,domain,__index_level_0__,owner_repo
43820,CVE-2020-24345,2020-08-13T19:15:13.807,https://github.com/jerryscript-project/jerrysc...,"[Exploit, Issue Tracking, Patch, Third Party A...",github.com,155052,"(jerryscript-project, jerryscript)"
65210,CVE-2020-23308,2021-06-10T23:15:07.843,https://github.com/jerryscript-project/jerrysc...,"[Exploit, Issue Tracking, Patch, Third Party A...",github.com,225114,"(jerryscript-project, jerryscript)"
119776,CVE-2023-27119,2023-03-10T02:15:58.600,https://github.com/WebAssembly/wabt/issues/1990,"[Exploit, Issue Tracking]",github.com,392177,"(WebAssembly, wabt)"
80300,CVE-2021-36414,2022-01-10T23:15:09.223,https://github.com/gpac/gpac/issues/1840,"[Exploit, Issue Tracking, Patch, Third Party A...",github.com,275294,"(gpac, gpac)"
77447,CVE-2021-28236,2021-12-02T22:15:08.893,https://github.com/LibreDWG/libredwg/issues/324,"[Exploit, Third Party Advisory]",github.com,265944,"(LibreDWG, libredwg)"


In [8]:
print('filtered_references_df len:\t', len(filtered_references_df))

filtered_references_df len:	 1383


## Scrape issues from GitHub referenced Issues

In [9]:
import requests
import os
USER_MAIN = os.getenv("GITHUB_USER_MAIN")
USER_PRIV = os.getenv("GITHUB_USER_PRIV")
USER_SCHOOL = os.getenv("GITHUB_USER_SCHOOL")

API_KEY_MAIN = os.getenv("GITHUB_API_KEY_MAIN")
API_KEY_PRIV = os.getenv("GITHUB_API_KEY_PRIV")
API_KEY_SCHOOL = os.getenv("GITHUB_API_KEY_SCHOOL")
counter = 0
dict_auth = [
    {
        'user': USER_MAIN,
        'secret': API_KEY_MAIN

    },
    {
        'user': USER_PRIV,
        'secret': API_KEY_PRIV
    },
    {
        'user': USER_SCHOOL,
        'secret': API_KEY_SCHOOL
    }
]


def get_issue(url_html):
    
    url = url_html.replace("github.com", "api.github.com/repos")
    return _getter(url, url_html)


def _getter(url, html_url):
    global counter

    counter += 1
    auth = (dict_auth[counter % 3]['user'], dict_auth[counter % 3]['secret'])
    response = requests.get(url, auth=auth)

    if response.status_code != 200:
        print(f"Error for URL: {url}")
        print(f"Status Code: {response.status_code}")
        print(f"Response: {response.json()}")
        return  # Skip this URL
    
    res = response.json()
    try:
        yield {
            "url": html_url,
            "body": res['body'],
            "title": res['title'],
            "comments_url": res['comments_url'],
            "comments_count": res['comments'],
            "created_at": res['created_at'],
            "updated_at": res["updated_at"],
            "html_url": res["html_url"],
            "github_id": res["id"],
            "number": res["number"]
    
        }
    except Exception as e:
        print(url)
        print(f"Exception: {e}")


In [10]:
data = []

for index, row in tqdm(filtered_references_df.iterrows()):
    for item in get_issue(row['url']):
        data.append(item)

197it [00:50,  4.63it/s]

Error for URL: https://api.github.com/repos/emlog/emlog/issues/62
Status Code: 410
Response: {'message': 'This issue was deleted', 'documentation_url': 'https://docs.github.com/rest/issues/issues#get-an-issue', 'status': '410'}


204it [00:52,  4.25it/s]

Error for URL: https://api.github.com/repos/emlog/emlog/issues/54
Status Code: 410
Response: {'message': 'This issue was deleted', 'documentation_url': 'https://docs.github.com/rest/issues/issues#get-an-issue', 'status': '410'}


234it [00:59,  4.98it/s]

...

Error for URL: https://api.github.com/repos/wqywfvc/CVE/issues/2
Status Code: 404
Response: {'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest', 'status': '404'}


1381it [06:04,  5.06it/s]

Error for URL: https://api.github.com/repos/wqywfvc/CVE/issues/10
Status Code: 404
Response: {'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest', 'status': '404'}
Error for URL: https://api.github.com/repos/wqywfvc/CVE/issues/11
Status Code: 404
Response: {'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest', 'status': '404'}


1383it [06:04,  3.79it/s]

Error for URL: https://api.github.com/repos/wqywfvc/CVE/issues/12
Status Code: 404
Response: {'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest', 'status': '404'}
Error for URL: https://api.github.com/repos/wqywfvc/CVE/issues/16
Status Code: 404
Response: {'message': 'Not Found', 'documentation_url': 'https://docs.github.com/rest', 'status': '404'}


In [11]:
display(pd.DataFrame(data).head())
print(len(pd.DataFrame(data)))

,url,body,title,comments_url,comments_count,created_at,updated_at,html_url,github_id,number
0,https://github.com/libming/libming/issues/169,I found a a heap-buffer-overflow problem in fu...,a heap-buffer-overflow problem in function wri...,https://api.github.com/repos/libming/libming/i...,1,2018-12-28T10:35:44Z,2019-01-07T16:23:12Z,https://github.com/libming/libming/issues/169,394605758,169
1,https://github.com/axiomatic-systems/Bento4/is...,A crafted input will lead to Memory allocation...,Allocate for large amounts of memory failed in...,https://api.github.com/repos/axiomatic-systems...,2,2018-12-31T15:52:26Z,2019-01-12T21:14:48Z,https://github.com/axiomatic-systems/Bento4/is...,394998583,350
2,https://github.com/axiomatic-systems/Bento4/is...,there is memory leaks in Ap4String.cpp\r\n\r\n...,memory leaks in Ap4String.cpp,https://api.github.com/repos/axiomatic-systems...,1,2019-01-10T17:00:31Z,2019-01-12T09:36:13Z,https://github.com/axiomatic-systems/Bento4/is...,397925621,357
3,https://github.com/axiomatic-systems/Bento4/is...,A crafted input will lead to failed allocate...,failed to allocate LargeMmapAllocator in Ap4A...,https://api.github.com/repos/axiomatic-systems...,1,2019-01-25T07:49:22Z,2019-12-26T14:50:59Z,https://github.com/axiomatic-systems/Bento4/is...,403042062,361
4,https://github.com/ImageMagick/ImageMagick/iss...,### Prerequisites\r\n\r\n- [x] I have written ...,memory leak in WritePSDChannel in coders/psd.c,https://api.github.com/repos/ImageMagick/Image...,2,2019-01-17T03:11:55Z,2019-02-12T01:45:47Z,https://github.com/ImageMagick/ImageMagick/iss...,400092278,1451


1347


In [12]:
issues_reference_df = pd.merge(
    filtered_references_df,
    pd.DataFrame(data),
    on='url'
)
print(len(issues_reference_df))

1347


In [13]:
issues_reference_df.head(1)

,id,published,url,tags,domain,__index_level_0__,owner_repo,body,title,comments_url,comments_count,created_at,updated_at,html_url,github_id,number
0,CVE-2019-3572,2019-01-02T15:29:00.330,https://github.com/libming/libming/issues/169,"[Exploit, Third Party Advisory]",github.com,82,"(libming, libming)",I found a a heap-buffer-overflow problem in fu...,a heap-buffer-overflow problem in function wri...,https://api.github.com/repos/libming/libming/i...,1,2018-12-28T10:35:44Z,2019-01-07T16:23:12Z,https://github.com/libming/libming/issues/169,394605758,169


In [14]:
non_filtered_count = len(issues_reference_df)
total_df = pd.merge(vulnerabilities_df, issues_reference_df, on='id')
total_df = total_df.drop('published_y', axis=1)
display(total_df.head(5))
non_na_count = len(total_df.dropna(subset='primary_cwe'))
print('non filtered issue count\t', non_filtered_count)
print('filtered issue count:\t', len(total_df))
print('non na issue count:\t', non_na_count)
print('percentage of filtered:\t', non_na_count / len(total_df))
print('percentage of non filtered:\t', non_na_count / non_filtered_count)

,id,published_x,descriptions,metrics,weaknesses,references,configurations,cwe_list,primary_cwe,url,...,owner_repo,body,title,comments_url,comments_count,created_at,updated_at,html_url,github_id,number
0,CVE-2019-3572,2019-01-02T15:29:00.330,An issue was discovered in libming 0.4.8. Ther...,"{'cvssMetricV2': [{'acInsufInfo': False, 'base...","[{'description': [{'lang': 'en', 'value': 'CWE...","[{'source': 'cve@mitre.org', 'tags': ['Exploit...",[{'nodes': [{'cpeMatch': array([{'criteria': '...,"[125, 787]",None,https://github.com/libming/libming/issues/169,...,"(libming, libming)",I found a a heap-buffer-overflow problem in fu...,a heap-buffer-overflow problem in function wri...,https://api.github.com/repos/libming/libming/i...,1,2018-12-28T10:35:44Z,2019-01-07T16:23:12Z,https://github.com/libming/libming/issues/169,394605758,169
1,CVE-2018-20659,2019-01-02T17:29:00.220,An issue was discovered in Bento4 1.5.1-627. T...,"{'cvssMetricV2': [{'acInsufInfo': False, 'base...","[{'description': [{'lang': 'en', 'value': 'CWE...","[{'source': 'cve@mitre.org', 'tags': ['Exploit...",[{'nodes': [{'cpeMatch': array([{'criteria': '...,[770],770,https://github.com/axiomatic-systems/Bento4/is...,...,"(axiomatic-systems, Bento4)",A crafted input will lead to Memory allocation...,Allocate for large amounts of memory failed in...,https://api.github.com/repos/axiomatic-systems...,2,2018-12-31T15:52:26Z,2019-01-12T21:14:48Z,https://github.com/axiomatic-systems/Bento4/is...,394998583,350
2,CVE-2019-6132,2019-01-11T05:29:01.763,An issue was discovered in Bento4 v1.5.1-627. ...,"{'cvssMetricV2': [{'acInsufInfo': False, 'base...","[{'description': [{'lang': 'en', 'value': 'CWE...","[{'source': 'cve@mitre.org', 'tags': ['Exploit...",[{'nodes': [{'cpeMatch': array([{'criteria': '...,[401],401,https://github.com/axiomatic-systems/Bento4/is...,...,"(axiomatic-systems, Bento4)",there is memory leaks in Ap4String.cpp\r\n\r\n...,memory leaks in Ap4String.cpp,https://api.github.com/repos/axiomatic-systems...,1,2019-01-10T17:00:31Z,2019-01-12T09:36:13Z,https://github.com/axiomatic-systems/Bento4/is...,397925621,357
3,CVE-2019-6966,2019-01-25T23:29:00.237,An issue was discovered in Bento4 1.5.1-628. T...,"{'cvssMetricV2': [{'acInsufInfo': False, 'base...","[{'description': [{'lang': 'en', 'value': 'CWE...","[{'source': 'cve@mitre.org', 'tags': ['Exploit...",[{'nodes': [{'cpeMatch': array([{'criteria': '...,[770],770,https://github.com/axiomatic-systems/Bento4/is...,...,"(axiomatic-systems, Bento4)",A crafted input will lead to failed allocate...,failed to allocate LargeMmapAllocator in Ap4A...,https://api.github.com/repos/axiomatic-systems...,1,2019-01-25T07:49:22Z,2019-12-26T14:50:59Z,https://github.com/axiomatic-systems/Bento4/is...,403042062,361
4,CVE-2019-7395,2019-02-05T00:29:00.400,"In ImageMagick before 7.0.8-25, a memory leak ...","{'cvssMetricV2': [{'acInsufInfo': False, 'base...","[{'description': [{'lang': 'en', 'value': 'CWE...","[{'source': 'cve@mitre.org', 'tags': ['Broken ...",[{'nodes': [{'cpeMatch': array([{'criteria': '...,[401],401,https://github.com/ImageMagick/ImageMagick/iss...,...,"(ImageMagick, ImageMagick)",### Prerequisites\r\n\r\n- [x] I have written ...,memory leak in WritePSDChannel in coders/psd.c,https://api.github.com/repos/ImageMagick/Image...,2,2019-01-17T03:11:55Z,2019-02-12T01:45:47Z,https://github.com/ImageMagick/ImageMagick/iss...,400092278,1451


non filtered issue count	 1347
filtered issue count:	 1261
non na issue count:	 1250
percentage of filtered:	 0.9912767644726408
percentage of non filtered:	 0.9279881217520416


In [17]:
total_df.columns

Index(['id', 'published_x', 'descriptions', 'metrics', 'weaknesses',
       'references', 'configurations', 'cwe_list', 'primary_cwe', 'url',
       'tags', 'domain', 'owner_repo', 'body', 'title', 'comments_url',
       'comments_count', 'created_at', 'updated_at', 'html_url', 'github_id',
       'number'],
      dtype='object')

In [16]:
total_df = total_df.drop(columns=['__index_level_0__'])
total_df.columns

Index(['id', 'published_x', 'descriptions', 'metrics', 'weaknesses',
       'references', 'configurations', 'cwe_list', 'primary_cwe', 'url',
       'tags', 'domain', 'owner_repo', 'body', 'title', 'comments_url',
       'comments_count', 'created_at', 'updated_at', 'html_url', 'github_id',
       'number'],
      dtype='object')

In [18]:
total_df = total_df.drop(columns=['weaknesses', ])

In [19]:
d = {
    'id': 'cve_id',
    'published_x': 'cve_published',
    'descriptions': 'cve_descriptions',
    'metrics': 'cve_metrics',
    'references': 'cve_references',
    'configurations': 'cve_configurations',
    'cwe_list': 'cve_cwe_list',
    'primary_cwe': 'cve_primary_cwe',
    'tags': 'cve_tags',
    'owner_repo': 'issue_owner_repo',
    'body': 'issue_body',
    'title': 'issue_title',
    'comments_url': 'issue_comments_url',
    'comments_count': 'issue_comments_count',
    'created_at': 'issue_created_at',
    'updated_at': 'issue_updated_at',
    'html_url': 'issue_html_url',
    'github_id': 'issue_github_id',
    'number': 'issue_number'
}
total_df =total_df.rename(columns=d)
display(total_df.sample(1))
len(total_df)

,cve_id,cve_published,cve_descriptions,cve_metrics,cve_references,cve_configurations,cve_cwe_list,cve_primary_cwe,url,cve_tags,...,issue_owner_repo,issue_body,issue_title,issue_comments_url,issue_comments_count,issue_created_at,issue_updated_at,issue_html_url,issue_github_id,issue_number
854,CVE-2023-33797,2023-05-24T20:15:10.893,A stored cross-site scripting (XSS) vulnerabil...,"{'cvssMetricV2': None, 'cvssMetricV30': None, ...","[{'source': 'cve@mitre.org', 'tags': ['Exploit...",[{'nodes': [{'cpeMatch': array([{'criteria': '...,[79],79,https://github.com/anhdq201/netbox/issues/12,"[Exploit, Issue Tracking]",...,"(anhdq201, netbox)",# Version: 3.5.1\r\n\r\n# Description\r\nAn au...,"Stored Cross Site Scripting Vulnerability in ""...",https://api.github.com/repos/anhdq201/netbox/i...,0,2023-05-18T16:34:34Z,2023-05-18T16:34:34Z,https://github.com/anhdq201/netbox/issues/12,1715930151,12


1261

In [26]:
total_df = total_df.dropna(subset='cve_primary_cwe')
display(total_df.columns)
len(total_df)

Index(['cve_id', 'cve_published', 'cve_descriptions', 'cve_metrics',
       'cve_references', 'cve_configurations', 'cve_cwe_list',
       'cve_primary_cwe', 'url', 'cve_tags', 'domain', 'issue_owner_repo',
       'issue_body', 'issue_title', 'issue_comments_url',
       'issue_comments_count', 'issue_created_at', 'issue_updated_at',
       'issue_html_url', 'issue_github_id', 'issue_number'],
      dtype='object')

1250

In [31]:
import datasets

dataset = datasets.Dataset.from_pandas(total_df)
dataset = ds.remove_columns(['__index_level_0__'])
dataset.push_to_hub("Eathus/github-issues-references")

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Eathus/github-issues-references/commit/097815ab096e6fe3d5c5c9b4e19477035fd7ab98', commit_message='Upload dataset', commit_description='', oid='097815ab096e6fe3d5c5c9b4e19477035fd7ab98', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Eathus/github-issues-references', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Eathus/github-issues-references'), pr_revision=None, pr_num=None)

## All issues of top 50 repos (negative + positive dataset)

In [32]:
import requests
import os

counter = 0
USER_MAIN = os.getenv("GITHUB_USER_MAIN")
USER_PRIV = os.getenv("GITHUB_USER_PRIV")
USER_SCHOOL = os.getenv("GITHUB_USER_SCHOOL")

API_KEY_MAIN = os.getenv("GITHUB_API_KEY_MAIN")
API_KEY_PRIV = os.getenv("GITHUB_API_KEY_PRIV")
API_KEY_SCHOOL = os.getenv("GITHUB_API_KEY_SCHOOL")
counter = 0
dict_auth = [
    {
        'user': USER_MAIN,
        'secret': API_KEY_MAIN

    },
    {
        'user': USER_PRIV,
        'secret': API_KEY_PRIV
    },
    {
        'user': USER_SCHOOL,
        'secret': API_KEY_SCHOOL
    }
]
    
def get_issues(username, repo):
    
    
    tmpl = "https://api.github.com/repos/{username}/{repo}/issues?state=all&page=1&per_page=100"
    url = tmpl.format(username=username, repo=repo)
    return _getter(url)


def _getter(url):
    global  counter

    counter += 1
    link = dict(next=url)
    while 'next' in link:
        auth = (dict_auth[counter%3]['user'], dict_auth[counter%3]['secret'])
        response = requests.get(link['next'], auth=auth, timeout=100)
    
        if response.status_code != 200:
            print(f"Error for URL: {url}")
            print(f"Status Code: {response.status_code}")
            print(f"Response: {response.json()}")
            return  # Skip this URL
    
        '''
        if response.status_code != 200:
            raise IOError(
                "Non-200 status code %r; %r; %r" % (
                    response.status_code, url, response.json()))
        '''
        for result in response.json():
            yield result
    
        link = _link_field_to_dict(response.headers.get('link', None))

def _link_field_to_dict(field):

    if not field:
        return dict()

    return dict([
        (
            part.split('; ')[1][5:-1],
            part.split('; ')[0][1:-1],
        ) for part in field.split(', ')
    ])


In [36]:
from time import sleep
import pickle
repos_lists = []

file_path = "tmp/neg_issues.pkl"
if os.path.exists(file_path):
    try:
        # Load the object from the pickle file
        with open(file_path, "rb") as file:
            issues = pickle.load(file)
        print("Data loaded successfully:")
    except pickle.PickleError as e:
        print(f"Error loading the pickle file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print(f"The file '{file_path}' does not exist. issues = empty dict")
    issues = {}
for owner, repo in repos_set:
    
    if (owner, repo) in issues : 
        continue
    repo_issues = []
    with tqdm(get_issues(owner, repo)) as pbar:
        for issue in pbar:
            if "pull_request" not in issue.keys():
                item = {
                    "issue_owner_repo": (owner, repo),
                    "issue_body": issue['body'],
                    "issue_title":issue['title'],
                    "issue_comments_url": issue['comments_url'],
                    "issue_comments_count": issue['comments'],
                    "issue_created_at": issue['created_at'],
                    "issue_updated_at": issue["updated_at"],
                    "issue_html_url": issue["html_url"],
                    "issue_github_id": issue["id"],
                    "issue_number": issue["number"],
                }
                repo_issues.append(item)
    issues[(owner, repo)] = repo_issues
    with open(file_path, "wb") as file:  # 'wb' mode writes in binary format
        pickle.dump(issues, file)
                
                
df = pd.DataFrame(sum(issues.values(), []))

Data loaded successfully:


130167it [18:06, 119.86it/s]
2548it [00:19, 132.26it/s]


In [43]:
df = pd.DataFrame(sum(issues.values(), []))
display(len(df))
display(df.head())

260782

,issue_owner_repo,issue_body,issue_title,issue_comments_url,issue_comments_count,issue_created_at,issue_updated_at,issue_html_url,issue_github_id,issue_number
0,"(HryspaHodor, CVE)",# Itsourcecode Vehicle Management System Proje...,Itsourcecode Vehicle Management System Project...,https://api.github.com/repos/HryspaHodor/CVE/i...,0,2024-06-20T06:07:24Z,2024-06-20T06:07:24Z,https://github.com/HryspaHodor/CVE/issues/7,2363597038,7
1,"(HryspaHodor, CVE)",# Itsourcecode Tailoring Management System Pro...,Itsourcecode Tailoring Management System Proje...,https://api.github.com/repos/HryspaHodor/CVE/i...,0,2024-06-18T06:06:04Z,2024-06-18T06:06:05Z,https://github.com/HryspaHodor/CVE/issues/6,2358990403,6
2,"(HryspaHodor, CVE)",# Itsourcecode Vehicle Management System Proje...,Itsourcecode Vehicle Management System Project...,https://api.github.com/repos/HryspaHodor/CVE/i...,0,2024-06-18T06:01:51Z,2024-06-18T06:01:51Z,https://github.com/HryspaHodor/CVE/issues/5,2358984521,5
3,"(HryspaHodor, CVE)","# Itsourcecode ""Loan Management System Project...","# Itsourcecode ""Loan Management System Project...",https://api.github.com/repos/HryspaHodor/CVE/i...,0,2024-06-18T05:57:17Z,2024-06-18T05:57:17Z,https://github.com/HryspaHodor/CVE/issues/4,2358978656,4
4,"(HryspaHodor, CVE)","# Itsourcecode ""Student Management System "" in...","Itsourcecode ""Student Management System "" in P...",https://api.github.com/repos/HryspaHodor/CVE/i...,0,2024-06-18T05:51:27Z,2024-06-18T05:51:27Z,https://github.com/HryspaHodor/CVE/issues/3,2358970853,3


In [44]:
import datasets

dataset = datasets.Dataset.from_pandas(df)
dataset.push_to_hub("Eathus/github-issues-negatives")

Uploading the dataset shards: 100%|██████████| 2/2 [00:20<00:00, 10.44s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Eathus/github-issues-negatives/commit/162569cb7ce0a805d0dafbc0fa81ac6bbb5e5cae', commit_message='Upload dataset', commit_description='', oid='162569cb7ce0a805d0dafbc0fa81ac6bbb5e5cae', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Eathus/github-issues-negatives', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Eathus/github-issues-negatives'), pr_revision=None, pr_num=None)